# Lab 08: Reinforcement Learning
- Thiết lập một bài toán đơn giản, huấn luyện Q-table và Deep Q-Agent giải bài toán


## Thiết lập môi trường

Bài toán (Slide 37 - Q-Learning của thầy Đăng): 
- Trong một cái sân có 5 ô, đánh số từ 0 đến 4, người chơi sẽ ở một trong 5 ô tại một thời điểm, trò chơi bắt đầu với người chơi ở ô 0.
- Tại một lượt, người chơi chọn một trong hai hành động "FORWARD" hoặc "BACKWORD":
    - "FORWARD": tiến về phía trước một ô, tức là đang ở ô i thì đến ô i + 1, nếu đang ở ô 5 thì vẫn ở lại ô;
    - "BACKWARD": trở về vị trí ô 0;
- Khi chọn xong hành đồng, người chơi cần tung một đồng xu với xác suất mặt ngửa là $\gamma$, khi bị mặt ngửa, người chơi phải chọn hành động ngược lại.
- Khi tiến vào ô 0, hoặc đang ở ô 0 và thực hiện BACKWARD, thì người chơi nhận được 2 điểm.
- Khi tiến vào ô 4, hoặc đang ở ô 4 và thực hiện FORWARD thì người chơi được 10 điểm
- Chơi trong tổng cộng $n=10000$ lượt.

In [ ]:
import random

class Environment():
    """Định nghĩa Môi trường giả lập của bài toán"""
    def __init__(self, gamma=0.05):
        self.gamma = gamma            ## xác suất phải thực hiện hành động ngược lại 
        self.rewards = [2,0,0,0,10]   ## points cho các ô
        self.cur_pos = 0              ## vị trí hiện tại của người chơi
        
    def reset(self):
        ## Reset môi trường về trạng thái ban đầu, tức là về lại vị trí 0
        self.cur_pos = 0    
        return self.cur_pos
    
    def step(self, action):
        """Thực hiện hành động action với môi trường, action là str có giá trị 1 trong 2 "backward" và "forward"
        trả về quan sát (vị trí hiện tại), và điểm nhận được 
        """
        reward = 0
        flip = random.random() <= self.gamma    ##thả đồng xu xem có ngửa không
        if flip:      ## phải làm ngược lại
            if action == "backward":
                action = "forward"
            elif action == "forward":
                action = "backward"
            else:
                raise Exception("error value")
                
        if action == "backward":   ## quay lại ô 1
            self.cur_pos = 0
        elif action == "forward":  ## tiến 1 ô 
            if self.cur_pos < 4:
                self.cur_pos += 1
        reward = self.rewards[self.cur_pos]
        return self.cur_pos, reward

    
def play(agent, n_steps=10000):
    """truyền vào 1 agent để chơi trò chơi
    agent phải có phương thwusc take_action(obs), nhận vào quan sát và trả về hành động tiếp theo
    trả về tổng điểm agent đạt được sau n_steps lượt 
    """
    env = Environment()
    obs = env.reset()
    total_rewards = 0
    for step in range(n_steps):
        action = agent.take_action(obs)
        obs, reward = env.step(action)
        total_rewards += reward
    return total_rewards
    

### Random Agent
Thử chơi trò chơi với hành động được chọn ngẫu nhiên 50 50

In [ ]:
import random
class RandomAgent():
    
    def take_action(self, obs):
        if random.random() <= 0.5:
            return "backward"
        return "forward"
    
agent = RandomAgent()
score = play(agent)
print("Final Score: ", score)

## Naive Learning
Trong phần này ta sẽ huấn luyện một agent sử dụng Q-table để chơi.

Trước tiên thiết lập Q-agent trước

In [ ]:
index_to_action = {   ## số hóa hành động
    0: "forward",    
    1: "backward",
}

action_to_index = {
    'forward': 0,
    'backward': 1,
}

class QAgent():
    """Lớp định nghĩa Agent dùng 1 Q-table cho trước để chơi
    """
    def __init__(self, Qtable):
        """Nhận vào 1 Q-table,
        Bài toán có 2 hành động nên và 5 quan sát (5 vị trí) 
        Q-table là một ma trận 2 x 5
        """
        self.Qtable = Qtable
    
    def take_action(self, obs):
        """Nhận một quan sát và trả về hành động
        """
        action = np.argmax(self.Qtable[:, obs]) ## Với 1 quan sát, xem hành động nào có Q-value lớn nhất thì chọn hành động đó
        return index_to_action[action] 

Bây giờ ta sẽ học Q-table một cách ngây thơ :
- Chọn hành động tốt nhất dựa vào những gì đã học
- Nếu điểm bằng nhau thì chọn ngẫu nhiên
- Q-table cập nhất với discount = 0 (không xem xét tương lai)

In [ ]:
## Q-table
import numpy as np
Qtable = np.zeros((2, 5))



env = Environment()   ##tạo môi trường giả lập
obs = env.reset()     ## lấy quan sát đầu tiên

for episode in range(100000):    ## train trong 100000 vòng lặp 
    if Qtable[0,obs] == Qtable[1,obs]:      ##nếu 2 action cho quan sát cùng điểm thì chọn đại
        action_index = random.randint(0,1)   
    else:
        action_index = np.argmax(Qtable[:, obs])  ## không thì chọn action có điểm tốt nhất mà chọn 
    new_obs, reward = env.step(index_to_action[action_index])          ## cho hành động vào giả lập và nhận lại quan sát mới và điểm thưởng
    Qtable[action_index, obs] += reward   ## cập nhận lại Q-table cho hành động 
    obs = new_obs
print(Qtable)

In [ ]:
agent = QAgent(Qtable)
score = play(agent)
print("Final Score: ", score)

## Q learning
Giá trị Q-value được cập nhật như sau:
$$Q(s_t, a_t) = Q(s_t, a_t) + \alpha [r_{t+1} + \lambda \max_{a}Q(s_{t+1}, a) - Q(s_t, a_t)]$$
với $\alpha$ là learning rate, $\lambda$ là discount rate, $s_t$ là quan sát thời điểm $t$ và $r_{t+1}$ là phần thưởng sau khi thực hiện hành động $a_t$ với quan sát $s_t$.

Ngoài ra agent sẽ ngẫu nhiên thực hiện exploration với xác suất nào đó

In [ ]:
## Q-table

lrn_rate = 0.1     ## tham số cho Q-learning 
discount = 0.95

import numpy as np
Qtable = np.zeros((2, 5))
env = Environment()
obs = env.reset()
max_episodes = 100000

def do_exploration(cur_ep, max_eps):    ## trả về xem có thực hiện exploration không 
    return random.random() <= 0.5   ##xác suất 0.5

## Q-learning
for episode in range(max_episodes): ## episode là mỗi lần update Qtable
    if Qtable[0,obs] == Qtable[1,obs]: 
        action_index = random.randint(0,1)
    else:
        action_index = np.argmax(Qtable[:, obs])
        if do_exploration(episode, max_episodes):  ## nếu thực hiện exploration thì làm hành động ngược lại 
            action_index = 1 - action_index
    
    new_obs, reward = env.step(index_to_action[action_index])  ## cho vào chạy giả lập
    
    
    Qtable[action_index, obs] += lrn_rate*(reward + discount*np.max(Qtable[:, new_obs]) - Qtable[action_index, obs])
    ## cập nhật lại Q-table 
    
    
    obs = new_obs
print(Qtable)

In [ ]:
agent = QAgent(Qtable)
score = play(agent)
print("Final Score: ", score)

## Deep Q-learning

Bây giờ ta sẽ thay Q-table bằng một mạng neuron:
- Mạng neuron nhận vào một quan sát, và trả về Q-value của các hành động
- Input của mạng là 1 vector one-hot thể hiện vị trí hiện tại
- Output là một vector 2 chiều, thể hiện q-value cho 2 hành động

Với mỗi quan sát $s$ và hành động $a$, ta cố huấn luyện mạng sao cho q-value tiến tới
$$Q^*(s,a) = R_{s,a} + \gamma \max_{a'} Q(s',a')$$
với $\gamma$ là discount rate, $s'$ và $R_{s,a}$ là là quan sát và phần thưởng sau khi thực hiện hành động $a$ với quan sát $s$;

Với $Q*(s,a)$ là mục tiêu của $Q(s,a)$ mỗi vòng lặp, làm loss sẽ là
$$\frac{1}{m}\sum_{s,a} (Q(s,a) - Q^*(s,a))^2$$
tuy nhiên do mạng nơ-ron $nn$ nhận $s$ trả về $a$ nên hàm loss:
$$\frac{1}{m}\sum_{(s,a)} (nn_a(s) - nn_a^*(s))^2$$
với chỉ số $a$ là lấy giá trị tương ứng với $a$ trong vector output, $nn_a^*$ là giá trị cố định không dùng để học trong hàm loss.
Để code hàm loss này cần phải tự định nghĩa trong keras (xem code dưới)

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
index_to_action = {
    0: "forward",
    1: "backward",
}

class DeepQAgent():
    
    def __init__(self):
        
        ## build a neural network
        inputs = Input(shape=(5,))
        dense = Dense(units=10, activation='relu')(inputs)
        outputs = Dense(units=2)(dense)
        
        self.model = Model(inputs, outputs)  ##model input là 5, output là 2
        self.model.compile(optimizer='adam',
                          loss=self._loss_function)   ## hàm loss tự định nghĩa
     
    def _loss_function(self, target, output):
        """output là output của NN, là ma trận (batch_size, 2)
        output là 2 giá trị tuy nhiên chỉ có một giá trị trong hàm loss nên ta cần tách ra và bỏ giá trị ko cần khỏi loss
        ta quy định nhãn cho 1 sample có 2 giá trị, target q-value và action tương ứng"""
        actions = tf.cast(target[:, 0], tf.int32)    ## cột đầu quy định là action,
        actions = tf.one_hot(actions, depth=2, dtype=tf.float32)  ## chuyển action thành onehot
        target_values = target[:,1:]        ## lấy target value của action
        target_values = target_values*actions   ## đổi sang ma trận, (batch_size, 2), những chỗ không có trong loss có giá trị không
        return tf.reduce_mean(tf.square(target_values - output*actions))  
        ## tính mean square error, mấy chỗ không có trong loss đều là hằng 0 nên không được tính loss
    
    def _get_action_values(self, obs):
        """nhận 1 quan sát obs dưới dạng số nguyên, trả về vector q-values cho các hành động"""
        obs_onehot = np.zeros((1, 5))
        obs_onehot[0, obs] = 1        ##chuyển quan sát thành one-hot
        return self.model.predict(obs_onehot)
    
    def _take_action(self, obs):
        """nhận về quan sát obs, trả về action tốt nhất dạng index"""
        values = self._get_action_values(obs)
        return np.argmax(values, axis=-1)
    
    def take_action(self, obs):
        """nhận 1 quan sát, trả về hành động tốt nhất"""
        action = self._take_action(obs)[0]
        return index_to_action[action]
    
    
    

In [ ]:

agent = DeepQAgent()
print(agent.model.predict(np.eye(5)))  ## xem thửu q-table của agent hởi tạo ngẫu nhiên
score = play(agent)
print("Final Score: ", score)      ## chơi thử

In [ ]:
## Q-learning
def do_exploration():
    return random.random() <= 0.5

batch_size = 32
env = Environment(gamma=0.05)
obs = env.reset()
agent = DeepQAgent()
freezed_agent = DeepQAgent()
for episode in range(5000):    ## episode là mỗi lần update Qtable
    
    ##tạo sẵn numpy lưu obs và target q-value tương ứng
    memory_observations = np.zeros((batch_size, 5))   #lưu dạng one-hot để input vào model luôn
    target_Q_values = np.empty((batch_size, 2))  #label, cột đầu tiên lưu action, cột thứ 2 lưu target q-value
    
    if episode % 100 == 0:
        ## xem coi agent chơi sao 
        print(agent.model.predict(np.eye(5)).T)
        score = play(agent)
        print("Score at {}: {}".format(episode,score))
        
    if episode % 10 == 0:
        freezed_agent.model.set_weights(agent.model.get_weights())
        
    for m in range(batch_size):   ## vòng lặp này để lần lượt chạy giả lập obs và reward

        ## các bước chạy giả lập với obs
        action = agent.take_action(obs)     ## lấy hành động
        act_idx = action_to_index[action]   ## lấy index của hành độgn
        if do_exploration():                ## nếu exploration thì làm hành động ngược
            act_idx = 1-act_idx
            action = index_to_action[act_idx]
        new_obs, reward = env.step(action)          ## lấy reward của hành động
        values = freezed_agent._get_action_values(new_obs)  
        max_value = np.max(values)                  ## lấy max q-value của observation mới
        
        ##lưu obs, action và target q-value
        memory_observations[m,obs] = 1 
        target_Q_values[m, 0] = act_idx
        target_Q_values[m, 1] = reward + 0.95*max_value
        obs = new_obs
        
        
    ## train với input là các observations và label là action và target q-value của nó 
    agent.model.train_on_batch(memory_observations, target_Q_values)
    
    

In [ ]:
score = play(agent)
print("Final Score: ", score)

## Bài tập
Hãy tự định nghĩa một trò chơi, thiết lập giả lập environment và huấn luyện một deep Q-learning để chơi trò đó
- Trò chơi phức tạp xíu, nhưng đừng quá phức tạp lại fail mất, ít nhất là 20 observations và 2 actions nhé
- Mô tả trò chơi kĩ xíu, viết hẳn environment và comment vào
- Huấn luyện một deep Q-learning, lưu lại model để có gì anh chơi thử
- Thời gian làm bài cho bài thực hành này là 2 tuần